# The Countdown Show, Numbers-Game 
Countdown is a british show that broadcast on Channel 4. In the show, there are three games that are played in the show. We'll be concerned with the game that is called "Countdown" - Essentially, the contestants must use arithmetic mathematics to reach a random target number from six other numbers.

We'll be using the code available to us from the college labs, except the code will be refactored to use the functional approach.

In [194]:
# Import all the libraries that the lab used.
import itertools as it
import random
import operator
import pandas as pd

# Generate the list of the "large" numbers.
def generate_large(step = 25, amount = 4):
	res = []

	for i in range(1, amount + 1):
		res.append( i * step )

	return res

# Generating the small list, we'll use the same function we used to get the large numbers, except we'll manipulate (order and double the amount of them, not the value) the numbers
def generate_small():
	return sorted( generate_large(step = 1, amount = 10) * 2 )

def pick_random(list, amount = 1):
	return random.sample(list, amount)

def permutate(list, amount = 2):
	return it.permutations(list, amount)

def permutate_limited(list, amount, limit):
	return it.islice(permutate(list, amount), limit)

def numbers_game(large = None):
	large = large if large else random.randrange(0, 5)

	random_large = pick_random(generate_large(amount=large), amount = large)
	random_small = pick_random(generate_small(), amount = 6 - large)

	play_numbers = random_large + random_small

	target_number = random.randrange(101, 1000)

	return play_numbers, target_number



In [195]:
# Generate large numbers
large_numbers = generate_large(25, 4)
# Generate small numbers
small_numbers = generate_small()

# Fit that data into a pandas data frame
df = pd.DataFrame(large_numbers, columns = ['Large'])
# And finally display it
df

,Large
0,25
1,50
2,75
3,100


In [196]:
# Peek the small numbers as well, just to make sure we're correct. We'll peek only 5 of them, to keep this preview short, and to keep the notebook clean.
# Trust me, there's 20 elements in the list.
df = pd.DataFrame(small_numbers[:5], columns = ['Small'], )
df

,Small
0,1
1,1
2,2
3,2
4,3


In [197]:
# Or if you don't trust me, then see for yourself.
len(small_numbers)

20

In [198]:
play_numbers, target = numbers_game()

play_numbers, target

([50, 25, 9, 2, 10, 7], 967)

In [199]:
OP = [operator.add, operator.mul, operator.sub, operator.truediv]

target = max(play_numbers) * min(play_numbers);

def find_hits(play_numbers, target):
	hits = []

	for numbers, operator in it.product(permutate(play_numbers, 2), OP):
		if operator(numbers[0], numbers[1]) == target:
			print(numbers[0] , operator.__name__, numbers[1], '=', target)
			hits.append((numbers, operator))

	return hits

find_hits(play_numbers, target)

50 mul 2 = 100
2 mul 50 = 100


[((50, 2), <function _operator.mul(a, b, /)>),
 ((2, 50), <function _operator.mul(a, b, /)>)]